# Part 2

## 1. Import Embeddings and Dataset

In [1]:
# Import Embedding Matrix and Embedding Matrix's Train dataset vocab to index dictionary

from utils.file import load_from_local_file

embedding_matrix = load_from_local_file("models/embedding_matrix.pckl")
embedding_matrix_train_dataset_vocab_to_index: dict = load_from_local_file("models/embedding_matrix_train_dataset_vocab_to_index.pckl")

Loading object from local...
Object loaded from local!
Loading object from local...
Object loaded from local!


In [2]:
# Import Dataset
import pandas as pd

train_df = pd.read_csv("datasets/train.csv")
val_df = pd.read_csv("datasets/val.csv")
train_df.head()

,text,label
0,the rock is destined to be the 21st century's ...,1
1,"the gorgeously elaborate continuation of "" the...",1
2,effective but too-tepid biopic,1
3,if you sometimes like to go to the movies to h...,1
4,"emerges as something rare , an issue movie tha...",1


## 2. Train RNN Model

In [3]:
SEARCH_SPACE = {
  "batch_size": [50, 100, 200],
  "learning_rate": [0.005, 0.01, 0.025, 0.05],
  "optimizer_name": ["SGD", "Adagrad", "Adam", "RMSprop"],

  # RNN Model Parameters
  "hidden_dim": [16, 32, 64],
  "num_layers": [2, 4, 8, 16]
}

In [4]:
from models.RNN import RNN
import torch.nn as nn
import torch
from torch.utils.data import DataLoader
from solver import train, plot_loss_acc_graph
from utils.custom_dataset import TextDataset

def train_rnn_model_with_parameters(
    batch_size: int,
    learning_rate: float,
    optimizer_name: str,
    hidden_dim: int,
    num_layers: int,
):
  # Model
  model_rnn = RNN(
    embedding_matrix=embedding_matrix,
    hidden_dim=hidden_dim,
    num_layers=num_layers,
    output_dim=2
  )

  ########################
  ###### Parameters ######
  ########################
  batch_size = batch_size
  max_epochs = 10_000

  # SGD Optimizer
  learning_rate = learning_rate
  match optimizer_name:
    case "SGD":
      optimizer = torch.optim.SGD(model_rnn.parameters(), lr=learning_rate)
    case "Adagrad":
      optimizer = torch.optim.Adagrad(model_rnn.parameters(), lr=learning_rate)
    case "Adam":
      optimizer = torch.optim.Adam(model_rnn.parameters(), lr=learning_rate)
    case "RMSprop":
      optimizer = torch.optim.RMSprop(model_rnn.parameters(), lr=learning_rate)
    case _:
      raise Exception("Invalid optimizer name!")

  # Cross Entropy Loss 
  criterion = nn.CrossEntropyLoss()

  ########################
  ######## Dataset #######
  ########################
  train_dataset = TextDataset(
    dataframe=train_df,
    max_len=train_df["text"].str.split().apply(len).max(),
    embedding_matrix_vocab_to_index=embedding_matrix_train_dataset_vocab_to_index
  )
  train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)

  val_dataset = TextDataset(
    dataframe=val_df,
    max_len=val_df["text"].str.split().apply(len).max(),
    embedding_matrix_vocab_to_index=embedding_matrix_train_dataset_vocab_to_index
  )
  val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

  ########################
  ######### Train ########
  ########################
  model, avg_train_loss, avg_train_acc, avg_val_loss, avg_val_acc, num_of_epochs = train(
    model=model_rnn,
    criterion=criterion,
    optimizer=optimizer,
    train_dataloader=train_dataloader,
    val_dataloader=val_dataloader,
    max_epoch=max_epochs
  )

  ########################
  ######### Plot #########
  ########################
  subtitle = f"batch_size_{batch_size}; lr_{learning_rate}; optimizer_{optimizer_name}; hidden_dim_{hidden_dim}; num_layers_{num_layers}"
  save_filename_prefix = f"rnn/batch_size_{batch_size}-lr_{learning_rate}-optimizer_{optimizer_name}-hidden_dim_{hidden_dim}-num_layers_{num_layers}"
  
  # Plot Train Loss and Accuracy Graph
  plot_loss_acc_graph(
    loss_list=avg_train_loss, 
    acc_list=avg_train_acc, 
    dataset_type="train",
    subtitle=subtitle,
    save_filename_prefix=save_filename_prefix,
    display=False
  )

  # Plot Validation Loss and Accuracy Graph
  plot_loss_acc_graph(
    loss_list=avg_val_loss, 
    acc_list=avg_val_acc, 
    dataset_type="val",
    subtitle=subtitle,
    save_filename_prefix=save_filename_prefix,
    display=False
  )

  ########################
  ##### Return Value #####
  ########################
  configuration_results = {
    "batch_size": batch_size,
    "learning_rate": learning_rate,
    "optimizer_name": optimizer_name,

    # RNN Model Parameters
    "hidden_dim": hidden_dim,
    "num_layers": num_layers,

    # Model performance
    "train_loss": avg_train_loss[-1],
    "train_accuracy": avg_train_acc[-1],
    "val_loss": avg_val_loss[-1],
    "val_accuracy": avg_val_acc[-1],

    # Epoch Number
    "num_of_epochs": num_of_epochs
  }
  return configuration_results

[nltk_data] Downloading package punkt to C:\Users\Toh Jing
[nltk_data]     Qiang\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package treebank to C:\Users\Toh Jing
[nltk_data]     Qiang\AppData\Roaming\nltk_data...
[nltk_data]   Package treebank is already up-to-date!
[nltk_data] Downloading package punkt_tab to C:\Users\Toh Jing
[nltk_data]     Qiang\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [5]:
all_configurations_results = []

for hidden_dim in SEARCH_SPACE["hidden_dim"]:
  for num_layers in SEARCH_SPACE["num_layers"]:
    for optimizer_name in SEARCH_SPACE["optimizer_name"]:
      for batch_size in SEARCH_SPACE["batch_size"]:
        for learning_rate in SEARCH_SPACE["learning_rate"]:
          print(f"---------- batch_size_{batch_size}; lr_{learning_rate}; optimizer_{optimizer_name}; hidden_dim_{hidden_dim}; num_layers_{num_layers} ----------")
          configuration_results = train_rnn_model_with_parameters(
            batch_size=batch_size,
            learning_rate=learning_rate,
            optimizer_name=optimizer_name,
            hidden_dim=hidden_dim,
            num_layers=num_layers
          )
          all_configurations_results.append(configuration_results)

---------- batch_size_50; lr_0.005; optimizer_SGD; hidden_dim_16; num_layers_2 ----------


Epoch 11 (Val): 100%|██████████| 22/22 [00:00<00:00, 43.72it/s, acc=0.515, loss=0.764]


---------- batch_size_50; lr_0.01; optimizer_SGD; hidden_dim_16; num_layers_2 ----------


Epoch 11 (Val): 100%|██████████| 22/22 [00:00<00:00, 48.44it/s, acc=0.515, loss=0.863]


---------- batch_size_50; lr_0.025; optimizer_SGD; hidden_dim_16; num_layers_2 ----------


...

---------- batch_size_200; lr_0.025; optimizer_Adagrad; hidden_dim_16; num_layers_2 ----------


Epoch 11 (Val): 100%|██████████| 6/6 [00:00<00:00, 16.83it/s, acc=0.556, loss=0.687]


---------- batch_size_200; lr_0.05; optimizer_Adagrad; hidden_dim_16; num_layers_2 ----------


Epoch 2 (Train):  19%|█▊        | 8/43 [00:00<00:03, 10.38it/s, acc=0.625, loss=1.17]


KeyboardInterrupt: 

## 3. Model Configurations Comparison

In [9]:
model_configurations_results_df = pd.DataFrame.from_dict(all_configurations_results)
model_configurations_results_df.sort_values(by=["val_accuracy"], ascending=False).reset_index(drop=True)

,batch_size,learning_rate,optimizer_name,hidden_dim,num_layers,train_loss,train_accuracy,val_loss,val_accuracy,num_of_epochs
0,200,0.050,SGD,16,2,0.692284,0.565698,0.716110,0.555833,11
1,200,0.005,SGD,16,2,0.696331,0.519186,0.687343,0.555833,11
2,200,0.010,Adagrad,16,2,0.698037,0.519186,0.688852,0.555833,11
3,200,0.005,Adagrad,16,2,0.697021,0.495930,0.686907,0.555833,11
4,200,0.025,SGD,16,2,0.695644,0.542442,0.696105,0.555833,11
5,200,0.010,SGD,16,2,0.696223,0.519186,0.690097,0.555833,11
6,200,0.025,Adagrad,16,2,0.698202,0.495930,0.687463,0.555833,11
7,100,0.050,SGD,16,2,0.455782,0.798256,1.509111,0.515455,11
8,100,0.025,SGD,16,2,0.672918,0.588953,0.794848,0.515455,11
9,100,0.010,SGD,16,2,0.689455,0.554070,0.719162,0.515455,11


# a. Final Configuration of best model

# b. Accuracy on Testset

# c. Strategies to derive final sentence representation